In [1]:
! pip -q install -e ..

In [2]:
import weboftruth as wot

In [3]:
from os.path import join

from torch import cuda
from torch.optim import Adam 
import torch

import numpy as np
import pandas as pd

import torchkge
from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_fb15k237
from torchkge.utils.datasets import load_wikidatasets
from torchkge.data_structures import KnowledgeGraph

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.svm import SVC

import argparse
import os

path = '..'
wot.svo_data_path = join(path, 'data', 'SVO-tensor-dataset')
wot.svo_paths = {k:join(wot.svo_data_path, str(k)) for k in [100, 80, 50]}
wot.models_path = join(path, 'models')
#print(wot.svo_paths)
#wot.reset_paths('..')
#print(wot.svo_paths)

def modelslist(module):
    return [x for x in dir(module) if 'model' in x.lower()]

def exp_obj(x):
    print([_ for _ in dir(x) if '__' not in _])

In [4]:
df = pd.DataFrame({'from': {0: 'a', 1: 'b', 2: 'c'},
 'to': {0: 'b', 1: 'c', 2: 'b'},
 'rel': {0: '1', 1: '2', 2: '3'}})
kg = KnowledgeGraph(df)

In [5]:
kg.relations

tensor([0, 1, 2])

In [6]:
[k for k in dir(kg) if '__' not in k]

['dict_of_heads',
 'dict_of_tails',
 'ent2ix',
 'evaluate_dicts',
 'get_mask',
 'get_sizes',
 'head_idx',
 'n_ent',
 'n_facts',
 'n_rel',
 'rel2ix',
 'relations',
 'sanity_check',
 'split_kg',
 'tail_idx']

In [7]:
kg.dict_of_heads

defaultdict(set, {(1, 0): {0}, (2, 1): {1}, (1, 2): {2}})

In [8]:
kg.rel2ix

{'1': 0, '2': 1, '3': 2}

In [9]:
kg.ent2ix

{'a': 0, 'b': 1, 'c': 2}

In [10]:
path = '/Users/aabir/Documents/research/weboftruth'
model_type = 'TransE'
epochs = 5
small = True
ts = 100

print(f"Path: {path}\nModel Type: {model_type}")
print(f"Epochs: {epochs}\nSmall: {small}")
print(f"Truth share: {ts}")
for f in os.listdir(wot.svo_paths[ts]):
    if 'train' in f: tr_fn = f
    if 'valid' in f: val_fn = f
    if 'test' in f: test_fn = f


Path: /Users/aabir/Documents/research/weboftruth
Model Type: TransE
Epochs: 5
Small: True
Truth share: 100


In [11]:

#dfs = wot.utils.read_data(tr_fn, val_fn, test_fn,
#                            wot.svo_paths[ts])
#dfs = [df.drop('true_positive', axis=1
#            ) if 'true_positive' in df.columns else df
#            for df in dfs ]
#tr_kg, val_kg, test_kg = (wot.utils.df_to_kg(df) for df in dfs)

tr_kg = kg

mod = wot.CustomTransModel(tr_kg, model_type=model_type,
                                traints=ts, ent_emb_dim=2, rel_emb_dim=2,
                                model_path = '../models/TransE_06')

 saving trainkg to ../models/TransE_06/train_TransE_kg.csv
Creating TransE in folder: ../models/TransE_06


In [12]:
mod.model_path

'../models/TransE_06'

In [13]:

mod.set_sampler(samplerClass=BernoulliNegativeSampler, kg=tr_kg)
mod.set_optimizer(optClass=Adam)
mod.set_loss(lossClass=MarginLoss, margin=0.5)
# Move everything to CUDA if available
if cuda.is_available():
    print("Using cuda.")
    cuda.empty_cache()
    cuda.init()
    mod.model.cuda()
    mod.loss_fn.cuda()
mod.train_model(10, tr_kg)
mod.validate(tr_kg)


 saving TransE to ../models/TransE_06/e=1ts=100_TransE_model.pt
 saving TransE to ../models/TransE_06/best_ts=100_TransE_model.pt
 saving TransE to ../models/TransE_06/e=10ts=100_TransE_model.pt



3.6832730770111084

In [14]:
ents, rels = mod.model.get_embeddings()

In [15]:
mod.trainkg.ent2ix

{'a': 0, 'b': 1, 'c': 2}

In [16]:
kg.relations, kg.ent2ix

(tensor([0, 1, 2]), {'a': 0, 'b': 1, 'c': 2})

In [17]:
for hr, t in [(k, v) for k, (v,) in kg.dict_of_tails.items()]:
    print(hr[0], hr[1], t)

0 0 1
1 1 2
2 2 1


In [18]:
wot.utils.kg_to_df(kg)

,from,to,rel
0,a,b,1
1,b,c,2
2,c,b,3


In [19]:
all_is = [int(d.split('_')[1]) for d in os.listdir('../models')
                        #all items in model path
                        if os.path.isdir(join('../models', d)
                        # that are directories
                        ) and f'TransE_' in d]
                        #and are of type self.model_type
print(all_is)

[3, 4, 5, 2, 1, 6]


In [20]:
i = [x for x in range(1, len(all_is)+2) if x not in all_is][0]

In [21]:
wot.load_kg('../models/TransE_06')

In [22]:

evl8 = wot.evaluator.Evaluator('../models/TransE_06', trainkg=kg)

x_tr, y_tr = evl8.get_svo_model_embeddings(filepath=None, sovs=[(0, 1, 0), (1, 2, 1), (2, 1, 2),
                                                       (0, 2, 0), (1, 0, 2), (2, 1, 0)], 
                                                 Ys = np.array([1, 1, 1, 0, 0, 0]))

x_te, y_te = evl8.get_svo_model_embeddings(filepath=None, sovs=[(0, 1, 0), (1, 2, 1),(0, 2, 1), (1, 2, 2)],
                                           Ys = np.array([1, 1, 0, 0]))

best_ts=100_TransE_model.pt
250


In [23]:
for predmodel in [LinearRegression, Ridge, SVC]:
    evl8.set_pred_model(predmodel)
    evl8.train_pred_model(x_tr, y_tr)
    evl8.evaluate_pred_model(x_te, y_te)

Train accuracy on LinearRegression(): 83.33333333333334 %
Test accuracy on LinearRegression(): 50.0 %
Train accuracy on Ridge(): 50.0 %
Test accuracy on Ridge(): 50.0 %
Train accuracy on SVC(): 100.0 %
Test accuracy on SVC(): 50.0 %
